In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os
import json
import pandas as pd
os.environ["CARTOPY_USER_BACKGROUNDS"] = "/Users/atharvas/Documents/FALL_2020/LING506/repo/BG"

In [58]:
all_tweets = pd.read_pickle("../data/cleanmoji.pkl")
all_tweets['created_at_date'] = pd.to_datetime(all_tweets['created_at'], infer_datetime_format=True) 
us_tweets = all_tweets[all_tweets['cc'] == "US"]
all_tweets['cc'].unique()



array(['US', 'BS', 'BR', 'DO', 'AR', 'AU', 'PR', 'NL', 'MX', 'GB', 'DE',
       'CL', 'TH', 'HT', 'VI', 'GH', 'KY', 'ES', 'GN', 'GR', 'JM', 'IN',
       'CA', 'FR', 'PH', 'SE', 'HN', 'PA', 'PE', 'CO', 'CR', 'ZA', 'IE',
       'ID', 'BD', 'JP', 'NG', 'MY', 'GP', 'CH', 'VE', 'SA', 'BE', 'PT',
       'CN', 'BB', 'LK', 'LB', 'DK', 'AT', 'VN', 'IT'], dtype=object)

In [24]:
# fig = plt.figure(figsize=(19.2, 10.8))
# ax = plt.axes(projection=ccrs.Mercator(central_longitude=-60,  
#                                        min_latitude=-30,
#                                        max_latitude=60))
# ax.background_img(name='BM', resolution='low')
# ax.set_extent([-150, 30, -30, 60], crs=ccrs.PlateCarree())



In [60]:
# us_tweets['Emoji_1']
# pd.to_datetime(all_tweets['created_at']) 

all_tweets['created_at_date'].dt.tz_convert('US/Eastern')

0       2012-10-25 15:18:02-04:00
1       2012-10-26 13:30:44-04:00
2       2012-10-27 08:04:31-04:00
3       2012-10-27 18:47:17-04:00
4       2012-10-28 10:21:02-04:00
                   ...           
92052   2012-10-28 18:13:58-04:00
92053   2012-10-28 18:13:59-04:00
92054   2012-10-28 18:14:01-04:00
92055   2012-10-28 18:14:01-04:00
92056   2012-10-28 18:14:01-04:00
Name: created_at_date, Length: 92057, dtype: datetime64[ns, US/Eastern]